# EDA (Exploratory Data Analysis)

In [ ]:
from ucimlrepo import fetch_ucirepo
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV  
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, precision_score, recall_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np
import pandas as pd
import os

In [ ]:
# fetch dataset
wine_quality = fetch_ucirepo(id=186)
# data (as pandas dataframes)
X = wine_quality.data.features
y = wine_quality.data.targets

In [ ]:
df = X.copy()
df['quality'] = y

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

Eliminamos los elementos duplicados ya que solo provocarán que los modelos tengan un mayor overfitting hacia esas clases repetidas

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

Eliminamos los outliers 

In [ ]:
X = df.drop('quality', axis=1)
y = df['quality']

# Eliminar los outliers de X
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1

X = X[~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)]

# Eliminar de y las filas que se eliminaron de X
y = y[y.index.isin(X.index)]

df = X.copy()
df['quality'] = y

In [ ]:
df.shape

A continuación, analizamos la distribución de las instancias de cada clase. De esta forma, podremos comprobar si las clases están desbalanceadas

In [ ]:
sns.countplot(x=df['quality'])
plt.title('Quality Count')
plt.xlabel('Quality Value')
plt.ylabel('Count')
plt.show()

print(df['quality'].value_counts())

Se explora la correlación entre las características

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pandas as pd

X = df.drop('quality', axis=1)
y = df['quality']

# Suponiendo que tienes un dataframe X con las características y un vector y con las etiquetas
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# Obtener la importancia de las características
importances = model.feature_importances_

# Crear un DataFrame con los resultados
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

# Ordenar por importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Visualizar la importancia de las características
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.title('Importancia de las Características')
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.show()

# Ver la lista de importancias
print(importance_df)


# Classification

## Random Forest

Hay correlaciones bastante altas en la matriz de correlaciones.
Teniendo en cuenta la importancia de cada una de las features, se han decidido eliminar las columnas "density" y "free_sulfur_dioxide".
Además, las features "sulphates", "residual_sugar" y "pH" tienen una correlación muy baja con la calidad del vino.

Eliminaremos las features correlacionadas y que tengan menor importancia.
También se eliminará "sulphates" ya que tiene poca importancia y correlación con la calidad del vino.


In [ ]:
rf_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

Además, de forma general, dividiremos el dataset en train y test.
Esto se hará para evaluar el entrenamiento del modelo.
Intentaremos que la proporción de clases sea la misma en ambos conjuntos.
Por eso se usa stratify.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(rf_df.drop('quality', axis=1), rf_df['quality'], test_size=0.2,stratify=rf_df['quality'],random_state=42)

En cada algoritmo hay ciertos hiperparámetros que se pueden ajustar para mejorar el rendimiento del modelo.
Para encontrar aquellos que mejoren el rendimiento del modelo, se probarán varias combinaciones.
Comprobando cuás de ellas maximiza ciertas métricas, como la precisión, el recall o el f1-score.


En este caso, el random forest tiene los siguientes hiperparámetros:

- `n_estimators`: número de árboles en el bosque
- `max_depth`: profundidad máxima de los árboles
- `min_samples_split`: número mínimo de muestras necesarias para dividir un nodo
- `min_samples_leaf`: número mínimo de muestras necesarias en un nodo hoja


In [ ]:
param_grid = {
    'n_estimators': [300, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# class_weights_dict = dict(zip(np.unique(y_train), class_weights))

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Una vez obtenidos los mejores hiperparámetros, se entrenará el modelo con el dataset de entrenamiento y se evaluará con el de test.
Se mostrarán las métricas obtenidas y se compararán con las obtenidas en el entrenamiento.
Usaremos la matriz de confusión para ver cómo se comporta el modelo en cada clase.


In [ ]:
# Modelo con los mejores hiperparámetros
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print('Precisión:', accuracy)

In [ ]:
# Matriz de confusión
rf_conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 6))
sns.heatmap(rf_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

## Logistic Regression


In [ ]:
lr_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = lr_df.drop('quality', axis=1)
y = lr_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Normalización de los datos, es importante para que el modelo converja más rápido y para que no haya features que tengan más peso que otras ya que las lleva a una escala común.
Sobre todo cuando estamos trabajando con modelos que usan la distancia euclídea, como la regresión logística.


In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

### Lasso

Para este modelo, el hiperparámetro que se puede ajustar es `C`, que es el inverso de la fuerza de regularización.

La diferencia entre Lasso y Ridge es que Lasso puede llevar a que algunos coeficientes sean 0, lo que puede ser útil para seleccionar características.

In [ ]:
param_grid = {
    'C': list(map(lambda x: x/100, range(1, 200, 1))),
}

lasso_model = LogisticRegression(penalty='l1', solver='saga')

grid_search = GridSearchCV(estimator=lasso_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring="f1_macro")
grid_search.fit(X_train, y_train)


In [ ]:
lasso_model = grid_search.best_estimator_
lasso_model.fit(x_train_scaled, y_train)
y_pred_lasso = lasso_model.predict(x_test_scaled)
print('Precisión Lasso:', accuracy_score(y_test, y_pred_lasso))


In [ ]:
lasso_conf_matrix = confusion_matrix(y_test, y_pred_lasso)
sns.heatmap(lasso_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión Lasso')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

### Ridge

Ridge es similar a Lasso, pero no lleva a que los coeficientes sean 0.
Lo que puede ser útil si no queremos eliminar características y por el contrario queremos tenerlas todas en cuenta aunque algunas tengan menos importancia.

In [ ]:
param_grid = {
    'C': list(map(lambda x: x/100, range(1, 200, 1))),
}

ridge_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(estimator=ridge_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)


In [ ]:
print(grid_search.best_params_)


In [ ]:
ridge_model = grid_search.best_estimator_

ridge_model.fit(x_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(x_test_scaled)
print('Precisión Ridge:', accuracy_score(y_test, y_pred_ridge))


In [ ]:
ridge_conf_matrix = confusion_matrix(y_test, y_pred_ridge)
sns.heatmap(ridge_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión Ridge')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

## Ensemble


A continuación se hace un ensemble de los siguientes modelos: Random Forest, Linear Regression (Lasso) y Linear Regression (Ridge).

Es útil ya que se pueden combinar modelos que tengan diferentes fortalezas y debilidades.
Por lo que se puede obtener un modelo más robusto y generalizable, que no dependa tanto de las características de un solo modelo lo que nos dará un mejor rendimiento en general.


In [ ]:
ensemble_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = ensemble_df.drop('quality', axis=1)
y = ensemble_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Una posibilidad es usar un modelo de votación, que consiste en que cada modelo vote por la clase que cree que es la correcta y se elige la clase que más votos tenga.
A este se le puede añadir un peso a cada modelo, para que no todos tengan el mismo peso en la decisión final.
En nuestro caso, se le dará el mismo peso a cada modelo, ya que sólo usaremos 3 modelos y el aumentar el peso de uno de ellos puede llevar a un sobreajuste hacia ese modelo.


In [ ]:
from sklearn.ensemble import VotingClassifier

voting_model = VotingClassifier(estimators=[('lasso', lasso_model), ('ridge', ridge_model), ('random_forest', best_rf)], voting='soft')
voting_model.fit(X_train, y_train)

y_pred_voting = voting_model.predict(X_test)
print('Precisión Voting:', accuracy_score(y_test, y_pred_voting))

Otra posibilidad es usar un modelo de stacking, que consiste en que un modelo se entrene con las predicciones de los otros modelos y las características originales.
De esta forma, el modelo final puede aprender a combinar las predicciones de los otros modelos de una forma más óptima.
En nuestro caso, se usará un modelo de regresión logística como modelo final, ya que es un modelo sencillo y rápido de entrenar.
También se podría usar un modelo más complejo y hacer una búsqueda de hiperparámetros para encontrar aquellos que maximicen el rendimiento del modelo final.


In [ ]:
from sklearn.ensemble import StackingClassifier

stacking_model = StackingClassifier(estimators=[('lasso', lasso_model), ('ridge', ridge_model), ('random_forest', best_rf)], final_estimator=LogisticRegression())
stacking_model.fit(X_train, y_train)

y_pred_stacking = stacking_model.predict(X_test)
print('Precisión Stacking:', accuracy_score(y_test, y_pred_stacking))

## Boosting

Boosting puede ayudarnos a clasificar mejor las clases que están desbalanceadas, cosa que ocurre en nuestro dataset.
Además, puede ayudar a mejorar el rendimiento del modelo, ya que se entrenan varios modelos secuencialmente y cada uno se entrena para corregir los errores del anterior.

Para nuestro problema, probaremos con AdaBoost, GradientBoosting y HistGradientBoosting.
Las diferencias entre ellos son las siguientes:

- AdaBoost: entrena varios modelos secuencialmente, cada uno se entrena para corregir los errores del anterior.
- GradientBoosting: entrena varios modelos secuencialmente, cada uno se entrena para corregir los errores del anterior, pero en este caso se entrena un árbol de decisión en cada iteración.
- HistGradientBoosting: es similar a GradientBoosting, pero en este caso se usa un histograma para acelerar el entrenamiento.

Cada uno de estos modelos tiene hiperparámetros que se pueden ajustar para mejorar el rendimiento del modelo.
Por ejemplo, el número de estimadores además de los hiperparámetros de los árboles de decisión.

Se podrían hacer búsquedas de hiperparámetros para encontrar aquellos que maximicen el rendimiento del modelo.
No se hará ya que se ha visto cómo se haría en el apartado de [random forest](#Random-Forest).


In [ ]:
boosting_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = boosting_df.drop('quality', axis=1)
y = boosting_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
rf_base = RandomForestClassifier(n_estimators=100, random_state=42)
ada_boost = AdaBoostClassifier(estimator=rf_base, n_estimators=50, random_state=42)
gradient_boost = GradientBoostingClassifier(n_estimators=45, random_state=42)
hist_gradient_boost = HistGradientBoostingClassifier(max_iter=15, random_state=42)

In [ ]:
ada_boost.fit(X_train, y_train)
gradient_boost.fit(X_train, y_train)
hist_gradient_boost.fit(X_train, y_train)

In [ ]:
y_pred_ada = ada_boost.predict(X_test)
y_pred_gradient = gradient_boost.predict(X_test)
y_pred_hist_gradient = hist_gradient_boost.predict(X_test)

In [ ]:
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print('Precisión AdaBoost:', accuracy_ada)

accuracy_gradient = accuracy_score(y_test, y_pred_gradient)
print('Precisión GradientBoost:', accuracy_gradient)

accuracy_hist_gradient = accuracy_score(y_test, y_pred_hist_gradient)
print('Precisión HistGradientBoost:', accuracy_hist_gradient)

Cabe destacar que, aunque no se haya hecho una búsqueda de hiperparámetros, han dado unos resultados parecidos a los obtenidos en el apartado de [random forest](#Random-Forest).
Probablemente, si se hiciera una búsqueda de hiperparámetros, se podrían obtener mejores resultados.


In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = knn.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

## K-Nearest Neighbors


Es un algoritmo de aprendizaje supervisado que se puede usar tanto para clasificación como para regresión.
En caso de clasificación, se asigna la clase que más se repite entre los k vecinos más cercanos, mientras que en regresión se asigna la media de los k vecinos más cercanos.


Sólo tiene 3 hiperparámetros:

- `n_neighbors`: número de vecinos más cercanos
- `weights`: peso que se le da a los vecinos más cercanos
- `metric`: métrica que se usa para calcular la distancia entre las instancias


Se ve afectado por variables irrelevantes y por la escala de las variables.
Ambos ya los hemos tratado en los apartados anteriores.

In [ ]:
knn_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = knn_df.drop('quality', axis=1)
y = knn_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

Usando clasificación:


In [ ]:
param_grid = {
    'n_neighbors': list(range(1, 20, 1)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3, n_jobs=-1)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)


In [ ]:
# Modelo con los mejores hiperparámetros
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print('Precisión:', accuracy)

In [ ]:
# Matriz de confusión
knn_conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 6))
sns.heatmap(knn_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

Usando regresión:


In [ ]:
param_grid = {
    'n_neighbors': list(range(1, 20, 1)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
}

knn = KNeighborsRegressor()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)


In [ ]:
# Modelo con los mejores hiperparámetros
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Calcular el error
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")

Para visualizar los resultados, se ha usado la matriz de confusión redondeando las predicciones a la clase más cercana.


In [ ]:
y_pred_round = np.round(y_pred)

In [ ]:
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred_round)
print('Precisión:', accuracy)


In [ ]:
# Matriz de confusión
rf_conf_matrix = confusion_matrix(y_test, y_pred_round)

plt.figure(figsize=(10, 6))
sns.heatmap(rf_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

# Regression

## Random Forest Regressor


In [ ]:
rf_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(rf_df.drop('quality', axis=1), rf_df['quality'], test_size=0.2,stratify=rf_df['quality'],random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [300, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# class_weights_dict = dict(zip(np.unique(y_train), class_weights))

rf_regressor = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train, sample_weight=sample_weights)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

In [ ]:
# Modelo con los mejores hiperparámetros
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

In [ ]:
# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")

In [ ]:
y_pred_round = np.round(y_pred)

In [ ]:
# Matriz de confusión
rf_conf_matrix = confusion_matrix(y_test, y_pred_round)

plt.figure(figsize=(10, 6))
sns.heatmap(rf_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred_round)
print("Precisión:", accuracy)

## Multiple Regression

In [ ]:
lr_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(lr_df.drop('quality', axis=1), lr_df['quality'], test_size=0.2,stratify=rf_df['quality'],random_state=42)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")


## Stochastic Gradient Descent (SGD)

In [ ]:
sgd_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = sgd_df.drop('quality', axis=1)
y = sgd_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(max_iter=1000, loss='squared_error', random_state=42)

In [ ]:
sgd.fit(x_train_scaled, y_train)
y_pred = sgd.predict(x_test_scaled)

In [ ]:
# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")


# Try best features combination

In [ ]:
def classifier_algorithm_rf(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y,random_state=42)

    param_grid = {
        'n_estimators': [300, 500],
        'max_depth': [20, 30, 40],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [2, 4],
    }

    rf = RandomForestClassifier(random_state=42)

    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_rf = grid_search.best_estimator_
    y_pred = best_rf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    return accuracy, f1, precision, recall

def classifier_algorithm_lr(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y,random_state=42)

    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(X_train)
    x_test_scaled = scaler.transform(X_test)

    lasso_model = LogisticRegression(penalty='l1', solver='saga', C=0.1)
    lasso_model.fit(x_train_scaled, y_train)
    y_pred_lasso = lasso_model.predict(x_test_scaled)

    accuracy = accuracy_score(y_test, y_pred_lasso)
    f1 = f1_score(y_test, y_pred_lasso, average='weighted')
    precision = precision_score(y_test, y_pred_lasso, average='weighted')
    recall = recall_score(y_test, y_pred_lasso, average='weighted')
    return accuracy, f1, precision, recall

In [ ]:
import itertools

def combinacines_df(df):
    combinaciones = []
    for i in range(4, len(df.columns)+1):
        combinaciones.extend(list(itertools.combinations(df.columns, i)))
    return combinaciones

In [ ]:
def guardar_en_csv(combinacion,accuracy, f1, precision, recall,file_path):
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            f.write('combinacion,accuracy,f1,precision,recall\n')
    with open(file_path, 'a') as f:
        f.write(f'{combinacion},{accuracy},{f1},{precision},{recall}\n')

In [ ]:
combinaciones = combinacines_df(df.drop('quality', axis=1))

import warnings
warnings.filterwarnings('ignore')  # Ignora todos los warnings

for combinacion in combinaciones:
    X = df[list(combinacion)]
    y = df['quality']
    accuracy_lr, f1_lr, precision_lr, recall_lr = classifier_algorithm_lr(X, y)
    guardar_en_csv(combinacion,accuracy_lr, f1_lr, precision_lr, recall_lr,'resultados_lr.csv')

for combinacion in combinaciones:
    X = df[list(combinacion)]
    y = df['quality']
    accuracy_rf, f1_rf, precision_rf, recall_rf = classifier_algorithm_rf(X, y)
    guardar_en_csv(combinacion,accuracy_rf, f1_rf, precision_rf, recall_rf,'resultados_rf.csv')

# Clustering

In [ ]:
X = df.drop('quality', axis=1)
y = df['quality']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(data=X_pca, columns=['PCA1', 'PCA2', 'PCA3'])
pca_df['quality'] = y

In [ ]:
from plotly import express as px

fig = px.scatter_3d(
    pca_df, x='PCA1', y='PCA2', z='PCA3', 
    color='quality', color_continuous_scale='viridis',
    title='Reducción de Dimensionalidad con PCA (3D)',
    labels={'quality': 'Calidad', 'PCA1': 'Componente 1', 'PCA2': 'Componente 2', 'PCA3': 'Componente 3'}
)

# Mostrar el gráfico interactivo
fig.show()